In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import holoviews as hv
import pickle

In [ ]:
hv.extension("bokeh")

# Load data

In [ ]:
with open("all_data.pickle", "rb") as f:
    d = pickle.load(f)

In [ ]:
list(d.keys())

In [ ]:
measurements, regionprops, labels, img = d[
    "/n/scratch2/jqs1/fidelity/190301/jqs_photobleach_100ms_de32_mkate2_0000.nd2"
]

In [ ]:
regionprops.head()

In [ ]:
%%output size=150
#%%opts Image {+axiswise}
hv.Image(img / img.max()).options(cmap="gray") + hv.Image(labels != 0).options(
    cmap="blues"
)

# Plotting

In [ ]:
list(measurements.keys())

In [ ]:
%%output size=250
traces = measurements["mean"]
idxs = np.random.permutation(traces.shape[0])
downsample = (
    10  # set to 1 to show all traces (instead of 10%); this will make your browser slow
)
curves = [
    {"x": np.arange(traces.shape[1]), "y": traces[i], "i": idxs[i]}
    for i in range(traces.shape[0] // downsample)
]
hv.Contours(curves, vdims=["i"]).options(color_index="i", cmap="Category20", logy=True)

In [ ]:
%matplotlib inline
import pylab as plt
import pandas as pd
import numpy as np
from matplotlib import cm
from scipy.integrate import simps

# G = 10
df = pd.DataFrame(traces[:, 0:2000])  # + np.random.normal(0,1,Gsamp.T.shape))
df = df[df[0] > 1000]
bins = np.arange(df[0].min() - 1, df[0].max() + 1, 150)
df["bin"] = pd.cut(df[0], bins=bins)
p = df.apply(lambda x: x[:-1:] / x[0], axis=1)
p["bin"] = df["bin"]

tau = 30
pbar = p.groupby(p["bin"]).mean()[df.groupby(df["bin"]).size() > tau]
mu = df.groupby(df["bin"]).mean()[df.groupby(df["bin"]).size() > tau]
sigma2 = df.groupby(df["bin"]).var(ddof=0)[df.groupby(df["bin"]).size() > tau]

y = sigma2.div(mu[0].values, axis="rows")
x = pbar
r = y.values.flatten() / x.values.flatten()

linfit = np.polyfit(x.values.flatten(), y.values.flatten(), 1)
linfit_fn = np.poly1d(linfit)

plt.figure(figsize=(12, 8))
# plt.scatter(x,y)

xfit = np.linspace(0, 0.25, 100)
nu_bar = np.round(np.mean(r[np.isfinite(r)]), 2)
# plt.plot(xfit,nu_bar*xfit,'-r',linewidth=6)

plt.title(
    r"$\nu = \frac{\hat{\sigma}^2}{f_{max}\hat{p}(1-\hat{p})}$ =" + str(nu_bar),
    fontsize=20,
    pad=20,
)
plt.xlabel(r"$(1-\hat{p})$", fontsize=20)
plt.ylabel(r"$\frac{\hat{\sigma}^2}{f_{max}}$", fontsize=20)
bin_hist = df.groupby(df["bin"]).size()
print(bin_hist[bin_hist.values > tau])
print(r"mean ratio: ", np.mean(r[np.isfinite(r)]))
print("median ratio: ", np.median(r[np.isfinite(r)]))

nu_int = []

cmap = cm.get_cmap("coolwarm")
imax = x.index[-1].left
imin = x.index[0].left
for name, group in x.groupby("bin"):
    nu_int.append(-6 * simps(y.loc[name].values, x.loc[name].values))
    dp = x.loc[name].values
    dp = dp[x.loc[name].values < 1 / 2]
    f = y.loc[name].values[x.loc[name].values < 1 / 2]
    #     nu_int.append(-12*simps(f,dp))
    c = (name.left - imin) / (imax - imin)

    plt.scatter(1 - x.loc[name].values, y.loc[name].values, color=cmap(c), label=name)
    plt.legend()
print(nu_int, np.mean(nu_int))
plt.title(
    r"$\nu = 6\int\frac{\hat{\sigma}^2}{f_{max}}dp$ =" + str(np.mean(nu_int)),
    fontsize=20,
    pad=20,
)

In [ ]:
I0 = traces[:, 0]
plt.hist(I0, bins=30)
print(np.mean(I0), np.var(I0))